In [13]:
## update packages
# !pip install numpy pandas matplotlib seaborn scikit-learn lightgbm catboost xgboost --upgrade

In [24]:
## importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
## importing the necessary data
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

In [26]:
## Necessary functions

########## FUNCTIONS FOR INITIAL PROCESSING OF DATA ######################
## function to get the survey year
def get_survey_year(df):
    return pd.to_datetime(df["Survey_date"]).dt.year



## function to convert score ranges into equivalent grades
def scores_to_grades(df):
    cols = ["Math", "Mathlit", "Additional_lang", "Home_lang", "Science"]
    grade_dict = {
        "0 - 29 %" : "G",
        "30 - 39 %": "F",
        "40 - 49 %": "E",
        "50 - 59 %": "D",
        "60 - 69 %": "C",
        "70 - 79 %": "B",
        "80 - 100 %": "A"
    }
    
    for col in cols:
        for score_range in grade_dict.keys():
            df.loc[df[col] == score_range, col] = grade_dict[score_range]
            
            
            
## function to fill missing values with median
def fill_non_nan_median(df, col, filt_data):
    temp = df[df[filt_data[0]] == filt_data[1]][col]
    df.loc[(df[filt_data[0]] == filt_data[1]) & np.isnan(df[col]), col] = np.median(temp[~np.isnan(temp)])
    
    
            
## function to fill missing tenure values
def fill_tenure(df):
    df["Tenure_missing"] = 0
    df.loc[df["Tenure"].isna(), "Tenure_missing"] = 1
    df.loc[((df["Status"] == "studying") | (df["Status"] == "other")), "Tenure"] = 0
    fill_non_nan_median(df, "Tenure", ("Status", "wage employed"))
    fill_non_nan_median(df, "Tenure", ("Status", "unemployed"))
    fill_non_nan_median(df, "Tenure", ("Status", "self employed"))
    fill_non_nan_median(df, "Tenure", ("Status", "employment programme"))
    df.loc[df["Status"] == "wage and self employed", "Tenure"].fillna(df[df["Status"] == "wage and self employed"]["Tenure"].mean(), inplace=True)
    
    
    
## function to drop columns
def drop_cols(df, cols):
    df.drop(cols, axis = 1, inplace=True)
    
    
    
## function to engineer columns with missing values
def create_missing_feats(df, cols):
    for col in cols:
        missing_feat = f"{col}_missing"
        df[missing_feat] = 0
        df.loc[df[col].isna(), missing_feat] = 1

        
## function to missing feature values with mode
def fill_na_with_mode(df, cols):
    for col in cols:
        df[col].fillna(df[col].mode()[0], inplace=True)
        

        
## function to process data for training
def process_df(df):
    df = df.copy()
    df["Survey_year"] = get_survey_year(df)
    scores_to_grades(df)
    fill_tenure(df)
    df["Tenure"] = np.log1p(df["Tenure"]) ## taking care of skew
    create_missing_feats(df, ["Science", "Home_lang", "Mathlit",
                              "Math", "Degree", "Diploma",
                              "Schoolquintile", "Additional_lang", "Matric"
                             ])
    fill_na_with_mode(df, ["Matric", "Degree", "Diploma"])

    drop_cols(df, [
                    "Person_id", "Survey_date", "Science", "Home_lang", "Mathlit", "Math",
                     "Schoolquintile", "Additional_lang",
                  ]) 

    return df



########## FUNCTIONS FOR INITIAL PROCESSING OF MAKING PREDICTIONS ######################
## function for prediction on test data
def make_preds(data, model, savename):
    ids = data["Person_id"]
    data = data.drop("Person_id", axis = 1)
    data[["Tenure", "Round", "Survey_year", "Birthyear", "Birthmonth"]] = scaler.transform(data[["Tenure", "Round", "Survey_year", "Birthyear", "Birthmonth"]])
    data = pd.get_dummies(data, drop_first=True)
    preds = model.predict_proba(data)
    data_df = pd.DataFrame({"Person_id": ids.values, "Target": preds[:,1]})
    data_df.to_csv(f"./{savename}.csv", index = False)

def ret_preds(data, model):
    ids = data["Person_id"]
    data = data.drop("Person_id", axis = 1)
    data["Tenure"] = scaler.transform(test[["Tenure"]])
    data = pd.get_dummies(data, drop_first=True)
    preds = model.predict_proba(data)
    return preds

def disp_confmat(model, y_true, X):
    y_pred = model.predict(X)
    disp = ConfusionMatrixDisplay(confusion_matrix(y_true, y_pred))
    disp.plot()

In [27]:
## processing the training dataset
train = process_df(train)
train.to_csv("train_processed.csv", index=False)

# processing the test dataset
ids = test["Person_id"]
test = process_df(test)
test = pd.concat([ids,test], axis=1)
# test.to_csv("test_processed.csv", index=False)

##### Preparing Data for training

In [28]:
## importing necessary packages for training
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import xgboost

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [29]:
### preparing necessary variables for model training

## random state to use true out the model training
random_state = 42

## using stratifiedKFold to retain same proportions in train test splits as the original data
## during Grid search for hyperparameters
kf = StratifiedKFold(n_splits = 7, shuffle=True, random_state = random_state)

## Preparing scaler for normalization of numeric data
scaler = StandardScaler()
scaler.fit(train[["Tenure", "Round", "Survey_year", "Birthyear", "Birthmonth"]])

StandardScaler()

In [30]:
## Normalizing numeric features in training data
train[["Tenure", "Round", "Survey_year", "Birthyear", "Birthmonth"]] = scaler.transform(train[["Tenure", "Round", "Survey_year", "Birthyear", "Birthmonth"]])

## one-hot encoding of categorical features in training data
train = pd.get_dummies(train, drop_first=True)

In [31]:
## splitting data into features and targets
X = train.drop("Target", axis = 1)
y = train["Target"]

#### Prepating Best performing model for training

In [34]:
estimators = [
    ("xgbc", xgboost.XGBClassifier(learning_rate=0.07, max_depth= 5, n_estimators=400)),
    ("lgbc", LGBMClassifier(learning_rate=0.02, max_depth= 19, n_estimators=200)),
    ("cbc", CatBoostClassifier(learning_rate=0.03, max_depth=5, n_estimators=1400, verbose = 0))
]
meta_classifier = LogisticRegression(max_iter = 10000)

## Grid search hyperparameters for final estimator of stacking classifier
params = {
    "final_estimator__C": [0.9, 1]
    
}

## gridsearch object
grid_sc = GridSearchCV(StackingClassifier(estimators, final_estimator=meta_classifier, stack_method="predict_proba"),
                        params,
                        cv = kf,
                        scoring="roc_auc",
                        verbose = 10)
grid_sc.fit(X, y)

Fitting 7 folds for each of 2 candidates, totalling 14 fits
[CV 1/7; 1/2] START final_estimator__C=0.9......................................


C:\Python310\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Python310\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 937, number of negative: 2508
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000346 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 361
[LightGBM] [Info] Number of data points in the train set: 3445, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.271988 -> initscore=-0.984558
[LightGBM] [Info] Start training from score -0.984558


C:\Python310\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Python310\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)
C:\Python310\lib\site-packages\xgboost\data.py:33

C:\Python310\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Python310\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)


...

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[CV 7/7; 2/2] END .........final_estimator__C=1;, score=0.877 total time=  42.3s
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 1093, number of negative: 2927
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000417 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 363
[LightGBM] [Info] Number of data points in the train set: 4020, number of used features: 34
[LightGBM] [Info] [binary:Boo

C:\Python310\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Python310\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)
C:\Python310\lib\site-packages\xgboost\data.py:33

C:\Python310\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Python310\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 875, number of negative: 2341
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000356 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 359
[LightGBM] [Info] Number of data points in the train set: 3216, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.272077 -> initscore=-0.984110
[LightGBM] [Info] Start training from score -0.984110
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_le

GridSearchCV(cv=StratifiedKFold(n_splits=7, random_state=42, shuffle=True),
             estimator=StackingClassifier(estimators=[('xgbc',
                                                       XGBClassifier(base_score=None,
                                                                     booster=None,
                                                                     callbacks=None,
                                                                     colsample_bylevel=None,
                                                                     colsample_bynode=None,
                                                                     colsample_bytree=None,
                                                                     device=None,
                                                                     early_stopping_rounds=None,
                                                                     enable_categorical=False,
                                                                     eval_metric=None,
                                                                     feature_types=None,
                                                                     gam...
                                                                     n_jobs=None,
                                                                     num_parallel_tree=None,
                                                                     random_state=None, ...)),
                                                      ('lgbc',
                                                       LGBMClassifier(learning_rate=0.02,
                                                                      max_depth=19,
                                                                      n_estimators=200)),
                                                      ('cbc',
                                                       <catboost.core.CatBoostClassifier object at 0x00000142F9D4D270>)],
                                          final_estimator=LogisticRegression(max_iter=10000),
                                          stack_method='predict_proba'),
             param_grid={'final_estimator__C': [0.9, 1]}, scoring='roc_auc',
             verbose=10)

In [35]:
## printing out best AUC score obtained and associated final_estimator hyperparameter
print(grid_sc.best_score_)
print(grid_sc.best_params_)

0.8667087180539406
{'final_estimator__C': 0.9}


In [36]:
# make_preds(test, grid_sc, "temp-aft-update2")

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


C:\Python310\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Python310\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Python310\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)
